In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import math

% matplotlib inline

This class was originally seen on https://github.com/pytorch/pytorch/issues/1333 and I am just trying to understand it so that I will then be able to do my own

In [660]:
class CausalConv1d(nn.Conv1d):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1, groups=1, bias=True):
        super(CausalConv1d, self).__init__( in_channels, out_channels, kernel_size, stride=stride, padding=0,
            dilation=dilation, groups=groups, bias=bias)

        self.left_padding = dilation * (kernel_size - 1)

    def forward(self, input):

        temp = torch.unsqueeze(input, 0)

        x = F.pad(temp, (self.left_padding, 0, 0, 0))

        x = torch.squeeze(x, 0)
        
#         print("***************  WHEIGHT  ************")
#         print(self.weight)
        mask = torch.Tensor([[[1, 0]]])
        temp = self.weight.data * mask
        temp = torch.Tensor(temp)
        temp = nn.Parameter(temp, requires_grad = True)
#         print(temp)
        self.weight = self.weight
#         print(self.weight.requires_grad)
# #         self.weight.requires_grad = False
        print(self.weight)
#         print(Variable(torch.Tensor([1, 0])))
#         temp = nn.Parameter(torch.Tensor([[[1, 0]]]))
#         temp.requires_grad = True
#         print(temp.requires_grad)
#         self.weight.data.fill_(1.0, 0.0)
#         self.weight.requires_grad = True
        self.weight.data[0][0][1].fill_(0.0)
        print(self.weight)
# #         self.weight = nn.Parameter(torch.Tensor([[[2, 0]]]))
#         self.weight = nn.Parameter(torch.randn(1,1,2))
#         self.bias = nn.Parameter(torch.Tensor([0]))
        
        return super(CausalConv1d, self).forward(x)

In [661]:
causal = CausalConv1d(1, 1, 2)
print(causal)

CausalConv1d (1, 1, kernel_size=(2,), stride=(1,))


In [662]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(causal.parameters(), lr=0.001, momentum=0.9)

In [663]:
b = pd.read_csv("file_path.csv", header=None)

In [664]:
v = np.array(b)

In [665]:
running_loss = 0.0
i = 0
temp = []
for row in v:
    
    # get the inputs
    inputs = row[0:99].astype(np.float)
    target = row[99:100]

    # wrap them in Variable
#     target = Variable(torch.Tensor(target)).type(torch.LongTensor)
#     inputs = Variable(torch.Tensor([[inputs]]))
    
    target = Variable(torch.Tensor(target)).type(torch.LongTensor)
    inputs = Variable(torch.Tensor([[inputs]]))
    
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = causal(inputs)
    outputs = torch.squeeze(outputs, 0)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.data[0]
    if i % 100 == 99:
        temp.append(running_loss/100)
        print('loss: %.3f' % (running_loss / 100))
        running_loss = 0.0
        params = list(causal.parameters())
        print("***************  WHEIGHT  ************")
        print(len(params))
    i+=1
    
plt.plot(temp)
plt.show

print('Finished Training')

Parameter containing:
(0 ,.,.) = 
 -0.2444 -0.0316
[torch.FloatTensor of size 1x1x2]



AttributeError: 'float' object has no attribute 'fill_'

# Generating and saving sine data

In [ ]:
def SinGenerator():
    start = np.random.randint(-10, 10, size=1)
    x = np.linspace(start, start+2*np.pi, 100)
    sine = (np.sin(x) + 1 )* 30
    sine = np.ndarray.astype(sine, int)
    return sine

In [ ]:
sine = SinGenerator()
plt.plot(sine)
plt.show

In [ ]:
data = []
for _ in range(10000):
    data.append(SinGenerator())

In [ ]:
df = pd.DataFrame(data)
df.to_csv("file_path.csv", index=False, header=False)